# Census tract map of number of daily bus arrivals by CalEnviroScreen score

In [ ]:
import branca
import geopandas as gpd
import pandas as pd

import map_utils

from IPython.display import Markdown

pd.set_option("display.max_rows", 20)

In [ ]:
df = gpd.read_parquet("./bus_stop_times_by_tract.parquet")

df = df.assign(
    num_arrivals = df.num_arrivals.fillna(0),
    popdensity_group = pd.qcut(df.pop_sq_mi, q=3, labels=False) + 1,
    jobdensity_group = pd.qcut(df.jobs_sq_mi, q=3, labels=False) + 1,
    arrivals_group = pd.qcut(df.num_arrivals.fillna(0), q=3, labels=False) + 1,
)


def categorize_low_med_high(df, cols_to_avg):
    # Calculate average
    denom = len(cols_to_avg)
    
    df = df.assign(
        avg_density_score = round(df[cols_to_avg].sum(axis=1) / denom, 0)
     )
    
    return df

df = categorize_low_med_high(df, ["popdensity_group", "jobdensity_group"])

In [ ]:
sample_tracts = ["06001400100", "06001400600", "06001400700"]
to_plot = df[df.Tract.isin(sample_tracts)]

In [ ]:
# If equity_group==1, corresponds to lower CalEnviroScreen scores
# If equity_group==3, corresponds to higher CalEnviroScreen scores (worse off)

In [ ]:
popup_dict = {
    "Tract": "Census Tract",
    "equity_group": "CalEnviroScreen Group", 
    "avg_density_score": "Job/Population Density Group",
    "num_arrivals": "# Daily Bus Arrivals",
    "arrivals_group": "Arrivals Group",
}

tooltip_dict = {
    "Tract": "Census Tract",
    "equity_group": "CalEnviroScreen Group", 
    "avg_density_score": "Job/Population Density Group",
    "num_arrivals": "# Daily Bus Arrivals",
    "arrivals_group": "Arrivals Group",
} 

plot_col = "num_arrivals"
cut1 = df[df.arrivals_group==1][plot_col].max()
cut2 = df[df.arrivals_group==2][plot_col].max()
cut3 = df[df.arrivals_group==3][plot_col].max()

MIN_VALUE = df[plot_col].min()
MAX_VALUE = df[plot_col].max()

colormap_cutoff = [cut1, cut2, cut3]

#colorscale = branca.colormap.linear.Blues_09
colorscale = branca.colormap.StepColormap(
                colors=COLOR_LIST,
                #colors=[blue1, blue2, blue3,], 
                index=colormap_cutoff,
                vmin=MIN_VALUE, vmax=MAX_VALUE,
)

In [ ]:
#branca.colormap.ColorMap.
#branca.colormap.LinearColormap.
#branca.colormap.StepColormap.
display(branca.colormap.linear.Accent_03)
display(branca.colormap.step.Blues_03)

In [ ]:
print(branca.colormap.step.Blues_03.colors[0])

#https://matplotlib.org/stable/tutorials/colors/colormaps.html
#https://stackoverflow.com/questions/33596491/extract-matplotlib-colormap-in-hex-format


In [ ]:
from pylab import *
import matplotlib

cmap = cm.get_cmap('Wistia', 3)    # PiYG

COLOR_LIST = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    hex_value = matplotlib.colors.rgb2hex(rgba)
    print(hex_value)
    COLOR_LIST.append(hex_value)
    
    

In [ ]:
#https://stackoverflow.com/questions/61441415/python-folium-choropleth-map-legend-quantiles-overlap
to_plot = df[df.equity_group==1]

fig = map_utils.make_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = tooltip_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 500, fig_height = 600, 
                                    zoom=6, centroid = [35.8, -119.4])
                                    #zoom=12, centroid = [37.84, -122.27])

colorscale.add_to(fig)
fig.save("./img/fig1.html")

In [ ]:
#https://stackoverflow.com/questions/61441415/python-folium-choropleth-map-legend-quantiles-overlap
to_plot = df[df.equity_group==2]

fig = map_utils.make_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = tooltip_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 500, fig_height = 600, 
                                    zoom=6, centroid = [35.8, -119.4])

colorscale.add_to(fig)
fig.save("./img/fig2.html")

In [ ]:
#https://stackoverflow.com/questions/61441415/python-folium-choropleth-map-legend-quantiles-overlap
to_plot = df[df.equity_group==3]

fig = map_utils.make_choropleth_map(to_plot, 
                                    plot_col = plot_col, 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = tooltip_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 500, fig_height = 600, 
                                    zoom=6, centroid = [35.8, -119.4])

colorscale.add_to(fig)
fig.save("./img/fig3.html")

In [ ]:
# https://www.python-graph-gallery.com/292-choropleth-map-with-folium
#https://stackoverflow.com/questions/37466683/create-a-legend-on-a-folium-map

#https://stackoverflow.com/questions/47846744/create-an-asymmetric-colormap

In [ ]:
# Focus on high density areas, then map out low/med/high service
# For calenviroscreen worst ones, low/med/high service